# Avaliação Continuada 2 (AC2) - Dados de Treinamento

Abaixo, iremos criar um modelo para classificar mensagens de spam a partir de um dataset obtido no Kaggle.

## Imports

Primeiro, começamos a importar todas as bibliotecas que iremos precisar nesse notebook, apenas para simplificar mais tarde.

In [3]:
# Leitura dos módulos
from __future__ import print_function

# biblioteca usada para trabalhar com dataframes
import pandas as pd
from pandas import Series,DataFrame

# uma biblioteca usada para auxiliar a limpeza de tags html
import html

# bibliotecas básicas como numpy e matplotlib para exibir 
# alguns gráficos caso seja necessário. 
import numpy as np
import matplotlib.pyplot as plt

# biblioteca para usar para medir acuracia e separar os dados de treino e teste
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# biblioteca usada para criar modelos de word embedding
import gensim
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

np.random.seed(1000)

import re # biblioteca para expressoes regulares

import nltk # biblioteca para Processamento de Linguagem Natural
from nltk.stem.porter import PorterStemmer # para fazer a estemização em documentos da lingua inglesa
from nltk.stem import RSLPStemmer # para fazer a estemização em documentos da lingua portuguesa

Abaixo, um import que deve ser executado apenas um vez, caso não tenha executado, descomente a linha e execute novamente.

In [4]:
# nltk.download('all') # instala todos os recursos da biblioteca NLTK. Você pode descomentar esta linha na 1a vez que for executar este notebook

## Dataset

Depois, inicializamos o nosso dataset que foi baixado do Kaggle e pode ser obtido a partir [desse link](https://www.kaggle.com/team-ai/spam-text-message-classification).

In [5]:
data_set_url = "https://raw.githubusercontent.com/H4ad/h4ad.facens.artificial-inteligence.text-classification/master/datasets/spam-text-message-20170820.csv"
data_set = pd.read_csv(data_set_url,sep=",",header=0)

## Análise Exploratória

Abaixo, um pouco dos dados que iremos ver e tratar durante o notebook.

### O que há dentro?

Vamos começar primeiro a visualizar o que há dentro desse dataset com alguns comandos que nos mostram alguns informações básicas.

In [6]:
display(data_set.head(n=5))
display(data_set.describe())

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


,Category,Message
count,5572,5572
unique,2,5157
top,ham,"Sorry, I'll call later"
freq,4825,30


### Valores Duplicados

Na tabela acima, podemos ver que na coluna "Message", temos um total de 5572 valores mas apenas 5157 são únicos.

Para não influenciar ou causar algum problema durante os treinos, rodar um outro comando para realmente ter a contagem de quantos dados duplicados nós temos:

In [7]:
data_set.duplicated().sum()

415

O número é de 415, é bem expressivo. Dessa forma, vamos agora tratar de apagar esses valores duplicados.

In [8]:
print('Shape antes de remover os dados.')
print(data_set.shape)

data_set_cleaned = data_set.drop_duplicates(inplace=False)

print('\nShape após remover os dados.')
print(data_set_cleaned.shape)

print('\nQuantidade de itens removidos:', (data_set.shape[0] - data_set_cleaned.shape[0]))

Shape antes de remover os dados.
(5572, 2)

Shape após remover os dados.
(5157, 2)

Quantidade de itens removidos: 415


### Valores Faltando

Após checar valores duplicados, podemos também verificar se não há algum valor faltando, usando o comando abaixo:

In [9]:
data_set_cleaned.isna().sum() / data_set_cleaned.shape[0]

Category    0.0
Message     0.0
dtype: float64

E como podemos ver, tudo zero para ambas as colunas, dessa forma, não precisamos fazer mais nada já que não há valores nulos ou faltando.

## Pré-Processamento

Após essa analise inicial dos dados, vamos começar com a etapa de pré-processamento dos textos obtidos.

As técnicas e o código abaixo usado foi obtido e modificado a partir do código original escrito pelo professor Renato Moraes Silva que pode ser encontrado [nesse notebook.](https://facens.instructure.com/courses/7531/files/1356091?module_item_id=269232)


Sobre as técnicas, serão usadas as seguintes:

 - deixar todas as palavras com letras minúsculas
 - substituir os números pela tag *\<NUMBER\>*
 - substituir todas as URLS pela tag *\<URL\>*
 - substituir todos os emails pela tag *\<EMAIL\>*
 - substituir o símbolo de moeda pela tag *\<MONEY\>*
 - substituir todos os caracteres não-alfanuméricos por um espaço em branco

E o que foi adicionado além das técnicas acima foram:
 - remover as entidades do html (`&gt;` `&#62;`).

Além disso, será aplicado também um processo de *estemização*, no qual diminui uma palavra para o seu radical, tornando uma palavra como "flies" para "fli". 

E também, será aplicado um processo de remoção das *stopwords*, no qual remove palavras muito comuns de uma lingua. Como estamos usando um dataset em inglês, palavras como "i", "me", "my", "myself", "we" e muitas outras.

> Uma lista de exemplos mais detalhadas pode ser encontrado [nesse gist](https://gist.github.com/sebleier/554280).

In [10]:
def preprocessing(text, stemming = False, stopwords = False):
    """
    Funcao usada para fazer o tratamento de textos da lingua inglesa
    
    Parametros: 
        text: variavel do tipo string que contem o texto que devera ser tratado
        
        stemming: variavel do tipo booleana que indica se a estemizacao deve ser aplicada ou nao
        
        stopwords: variavel do tipo booleana que indica se as stopwords devem ser removidas ou nao
    """
    
    # Lower case
    text = text.lower()
    
    # remove tags HTML
    regex = re.compile('<[^<>]+>')
    text = re.sub(regex, " ", text) 
    
    # remove as entidades do HTML
    regex = re.compile('(&.+;)')
    text = re.sub(regex, " ", text) 
    
    # normaliza as URLs
    regex = re.compile('(http|https)://[^\s]*')
    text = re.sub(regex, "<URL>", text)

    # normaliza emails
    regex = re.compile('[^\s]+@[^\s]+')
    text = re.sub(regex, "<EMAIL>", text)
    
    #normaliza o símbolo de dólar
    regex = re.compile('[$]+')
    text = re.sub(regex, "<MONEY>", text)
    
    # converte todos os caracteres não-alfanuméricos em espaço
    regex = re.compile('[^A-Za-z0-9]+')  
    text = re.sub(regex, " ", text)
    
    # normaliza os numeros 
    regex = re.compile('[0-9]+')
    text = re.sub(regex, "<NUMBER>", text)
    
    # substitui varios espaçamentos seguidos em um só
    text = ' '.join(text.split())

    # remove stopwords
    if stopwords:
        words = text.split() # separa o texto em palavras
        words = [w for w in words if not w in nltk.corpus.stopwords.words('english')]
        text = " ".join( words )
    
    # aplica estemização
    if stemming: 
        stemmer_method = PorterStemmer()  
        words = text.split() # separa o texto em palavras
        words = [ stemmer_method.stem(w) for w in words ]
        text = " ".join( words )
    
    # remove palavras de apenas um caracter
    words = text.split() # separa o texto em palavras
    words = [ w for w in words if len(w)>1 ]
    text = " ".join( words )
    
    return text

Com a função acima, podemos agora chamar ela para realizar todo o pré-processamento do nosso dataset.

In [11]:
print("Um dos textos antes do processamento:")
display(data_set_cleaned["Message"][201])

data_set_processed = data_set_cleaned.copy()
data_set_processed["Message"] = data_set_processed["Message"].apply(preprocessing)

print("O mesmo texto, só que agora processado:")
display(data_set_processed["Message"][201])

Um dos textos antes do processamento:


'I sent you  &lt;#&gt;  bucks'

O mesmo texto, só que agora processado:


'sent you bucks'

## Word Embedding (Transformando texto em número)

Após realizar o pré-processamento, temos todos os nossos textos e palavras bem organizadas e formatadas. Dessa forma, o que devemos fazer é transformar essas palavras em um formato que o nosso modelo de decisão em arvore irá conseguir entender, e é ai que entra o Word Embedding.

Word Embedding é uma técnica para transformar um texto em um vetor de atributos com valores numericos. Há alguns pré-treinados, como o da Google, mas iremos criar um próprio porque os pré-treinados são gigantescos (na ordem de Gigas) e serão muito potentes para um simples experimento.

Assim, iremos começar agora a criar uma lista de lista com as palavras de cada mensagem no dataset.

In [12]:
sentences = data_set_processed["Message"].apply(lambda message: message.split()).tolist()

print("Algumas palavras que iremos usar para montar o nosso vocabulário:")
print(sentences[0])

Algumas palavras que iremos usar para montar o nosso vocabulário:
['go', 'until', 'jurong', 'point', 'crazy', 'available', 'only', 'in', 'bugis', 'great', 'world', 'la', 'buffet', 'cine', 'there', 'got', 'amore', 'wat']


E com essa lista de palavras, iremos agora treinar o modelo de Word Embedding utilizando a biblioteca [Genshim](https://radimrehurek.com/gensim/models/word2vec.html).

In [13]:
model = Word2Vec(sentences=sentences, vector_size=200, window=3, min_count=2, workers=1,seed=0)

Com o modelo treinado, podemos ver qual é o tamanho do vocabulário da seguinte forma:

In [14]:
print('Tamanho do Vocabulario: ', + len(model.wv))

Tamanho do Vocabulario:  3632


E se precisarmos obter o valor associado a cada palavra, podemos usar o seguinte código:

In [15]:
print(model.wv["call"])

[ 0.17725728 -0.12364591 -0.09128042  0.15352575  0.27465612 -0.34918335
  0.19557531  0.6514204  -0.18635306  0.23358381 -0.2508145  -0.3861254
  0.12062293  0.1948891  -0.09676713 -0.3436416  -0.08250508 -0.02306298
  0.03775226 -0.4997928   0.229319   -0.31791058 -0.17551939 -0.12988602
  0.07546745 -0.2335632  -0.05269529 -0.14021079 -0.34256917  0.13526137
  0.3936194   0.02491342  0.15715733 -0.07980499 -0.08084317  0.22652645
  0.27350834 -0.12216686 -0.06551714 -0.515588   -0.3249059  -0.09503899
 -0.01017304  0.11946314  0.4964906  -0.14012283 -0.21763206 -0.07942562
  0.16416462  0.22088058  0.02557784 -0.14922592 -0.22336413 -0.28794804
  0.07905076 -0.09045391 -0.00816449 -0.3156529  -0.3078355   0.12862799
  0.00405741  0.05194883  0.00328006 -0.07926335 -0.62332547  0.15191382
  0.00480628  0.68477184 -0.3179665   0.4928944  -0.0347524   0.00089589
  0.33645976  0.03821288  0.12150496  0.19722086  0.2751437  -0.2478644
 -0.49972895 -0.06039534 -0.21529883 -0.07922271 -0.4

E é possível até mesmo verificar a similaridade dessa palavra com outras, da seguinte forma:

In [16]:
print(model.wv.most_similar("call", topn=10))

[('claim', 0.9994459748268127), ('prize', 0.9993008971214294), ('<NUMBER>p', 0.9988323450088501), ('reply', 0.9988068342208862), ('free', 0.9987947940826416), ('cash', 0.9987867474555969), ('guaranteed', 0.9987627863883972), ('landline', 0.9987287521362305), ('stop', 0.9986884593963623), ('<NUMBER>', 0.9985957145690918)]


Dessa forma, agora que foi visto tudo que podemos fazer usando o modelo de Word Embedding, podemos usar ele para transformar todo o nosso dataset para um formato que será possível usar ele para o treinamento.

Vamos começar definindo uma função que irá transformar uma sentença inteira um vetor de números.

> A função abaixo é uma ligeira adaptação do código do professor Renato, e pode ser encontrado [nesse link.](https://facens.instructure.com/courses/7531/files/1356091?module_item_id=269232) 

In [17]:
def getDocvector(sentence):
  sentenceValues = []

  for word in sentence:
    # É interessante notar o uso do try aqui dentro,
    # ele se dá porque, durante a criação do modelo,
    # nós especificamos que ele só gere o modelo com
    # palavras que aparecem ao menos 2 vezes. Dessa 
    # forma, palavras com frequencia menor que isso
    # são simplesmente ignoradas, e ai podem causar 
    # que ao tentar acessar elas, ocorra uma exceção
    # porque ela não existe no modelo.
    try:
      sentenceValues.append(model.wv[word])
    except:
      pass

  if len(sentenceValues) > 0:
    return np.mean(sentenceValues, axis = 0)

  return np.zeros(model.wv.vector_size)

print('Testando com uma frase qualquer')
print(getDocvector(sentences[0]))

Testando com uma frase qualquer
[ 0.09666858 -0.07793677 -0.0642707   0.0669587   0.18798147 -0.20435497
  0.06802794  0.36140403 -0.11835702  0.10734684 -0.16304761 -0.20007738
  0.07085994  0.12438218 -0.04647421 -0.18095407 -0.06126513 -0.00562676
  0.01105956 -0.27157634  0.11489736 -0.1504359  -0.07379086 -0.09021427
  0.04434391 -0.1252791  -0.03314374 -0.08252606 -0.1887525   0.09725066
  0.21774572  0.00726212  0.06642633 -0.02066478 -0.03789778  0.12669984
  0.12304571 -0.07355275 -0.03274385 -0.307824   -0.15630648 -0.02252099
  0.00066224  0.06722633  0.2936538  -0.04929951 -0.11844446 -0.07437997
  0.09269835  0.1100501   0.02483305 -0.09935456 -0.11979835 -0.1545167
  0.03718428 -0.03136489 -0.01941977 -0.16338153 -0.16627374  0.05466416
  0.00815796  0.02251415 -0.00804271 -0.04178296 -0.33373347  0.0987446
  0.0119786   0.36427873 -0.1874299   0.29329967 -0.03065552  0.00333949
  0.19527505  0.01228885  0.0685506   0.10282546  0.15520123 -0.16496915
 -0.3098855  -0.01296

Com a função acima, podemos agora transformar todas as nossas mensagens do modelo para um vetores de valores, assim como, separar nossos valores de classificação e valores de treino e teste, da seguinte forma:

- x = Valores que podem ser usados para descrever se é spam ou não
- y = A informação se é realmente um spam

In [18]:
y = data_set_processed["Category"].to_numpy()
x = []

for sentence in data_set_processed["Message"]:
  x.append(getDocvector(sentence.split()))
  
x = np.array(x)

print("Shape do vetor de classe: ", y.shape)
print("Shape do vetor de valores: ", x.shape)

Shape do vetor de classe:  (5157,)
Shape do vetor de valores:  (5157, 200)


## Separando os Dados

Bom, após ter o modelo para poder transformar os textos em números, podemos agora começar a treinar o nosso modelo, mas antes, precisamos separar os dados que serão usados no treino.

Vamos separar os valores em:

- x_train = Valores de treino
- y_train = A classe se diz se é spam ou não de treino
- x_test = Valores para testar o modelo após o treino
- y_test = A classe dizendo se é spam ou não de teste para testar o modelo após o treino

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1000)

Além disso, vamos nos certificar que a tipagem dos valores estejam corretos, ou seja, todos sejam do tipo float.

In [20]:
x_train = x_train.astype(np.float64)
x_test = x_test.astype(np.float64)

Por fim, temos as seguintes métricas sobre os nossos dados de teste e treino.

In [21]:
print("Qntd. de amostras de treino: ", x_train.shape[0])
print("Qntd de amostras de teste: ", x_test.shape[0])

print("\nQtd. de dados de cada classe (treinamento)")
print("\tClasse ham: ", len(y_train[y_train == 'ham']))
print("\tClasse spam: ", len(y_train[y_train == 'spam']))

print("\nQtd. de dados de cada classe (teste)")
print("\tClasse ham: ", len(y_test[y_test == 'ham']))
print("\tClasse spam: ", len(y_test[y_test == 'spam']))

Qntd. de amostras de treino:  3609
Qntd de amostras de teste:  1548

Qtd. de dados de cada classe (treinamento)
	Classe ham:  3164
	Classe spam:  445

Qtd. de dados de cada classe (teste)
	Classe ham:  1352
	Classe spam:  196


## Tudo pronto

Dessa forma, com os dados do nosso dataset preparado, podemos agora ir para os outros notebooks para rodar os métodos de classificação.